In [52]:
import spacy
nlp = spacy.load('en')

In [89]:
s1 = "The Angry Man was awarded the second-best courtroom drama ever by the American Film Institute in April 19, 2012."
s2 = 'The Mission Impossible was nominated for Golden Globe Award.'
s3 = 'The GodFather won Oscar Award for best screenplay in 2012.'
s4 = 'The Mission Impossible was awarded Golden Globe Award.'
s5 = 'It was awarded the preservation in the U.S. National Film Registry of the Library of Congress in 1990.'
s6 = 'Godfather won the Oscars for Best Picture, Best Actor (Brando) and Best Adapted Screenplay (for Puzo and Coppola).'
s7 = 'The Angry Man was nominated for Academy Awards in the categories of Best Director, Best Picture, and Best Writing of Adapted Screenplay.'


In [90]:
sentence = s7
parse(sentence)

Matching with wriiten template: for The Angry Man was nominated for Academy Awards in the categories of Best Director, Best Picture, and Best Writing of Adapted Screenplay.
Has valid verb:True
Movie name : The Angry Man
Recognizer name :  <no-match>
Recognized As :  Academy Awards
nominate
Type recognition :  nomination
Time :  <no-match>


In [92]:
def parse(statement1):
    isValidForRecognizedAsTemplate = checkRecognizedAsTemplateEligibility(statement1)
    if(isValidForRecognizedAsTemplate):

        print("--------- Extracted Information -----------")
            
        movieName = extractMovieName(statement1)
        print("Movie name : " + str(movieName))

        recognizerName = extractRecognizerName(statement1)
        print("Recognizer name : " , recognizerName)

        recognizedAs = extractRecognizedAs(statement1)
        print("Recognized As : " , recognizedAs)

        typeOfRecognition = extractTypeOfRecoginition(statement1)
        print("Type recognition : " , typeOfRecognition)

        year = extractYear(statement1)
        print("Time : " , year)
    else:
        print(" NO - PARSE")

In [58]:
# statement1 = 'The Mission Impossible was nominated for Golden Globe Award.'
# statement1 = 'The Mission Impossible won Golden Globe Award.'
# statement1 = 'The Mission Impossible was awarded Golden Globe Award.'
statement1 = 'The Angry Man was awarded the second-best courtroom drama ever by the American Film Institute in April 19, 2012.'

isValidForRecognizedAsTemplate = checkRecognizedAsTemplateEligibility(statement1)
if(isValidForRecognizedAsTemplate):
    
    movieName = extractMovieName(statement1)
    print("extracted movie name is:" + str(movieName))
    
    recognizerName = extractRecognizerName(statement1)
    print("extracted recognizer name is:" , recognizerName)
    
    recognizedAs = extractRecognizedAs(statement1)
    print("extracted recognized As is:" , recognizedAs)
    
    typeOfRecognition = extractTypeOfRecoginition(statement1)
    print("extracted type recognition is:" , typeOfRecognition)
    
    year = extractYear(statement1)
    print("extracted year is:" , year)



Matching with wriiten template: for The Angry Man was awarded the second-best courtroom drama ever by the American Film Institute in April 19, 2012.
Has valid verb:True
extracted movie name is:The Angry Man
extracted recognizer name is: the American Film Institute
extracted recognized As is: the second-best courtroom drama
extracted type recognition is: winner
extracted year is: April 19, 2012


In [59]:
def extractMovieName(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    movieName = "<uninitialized>"
    
#     handle when release is with premiered.
    '''
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to : "+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    #print("vt:"+verbText)

    movieName = getMovie(statement,verbText)
    
    return movieName

In [60]:
def extractRecognizerName(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    writerName = "<uninitialized>"
    
#     handle when release is with premiered.
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        #print("verbTextChanged to"+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)

    #print("vt:"+verbText)

    writerName = getWriter(statement,verbText)
    
    return writerName

In [61]:
def extractRecognizedAs(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    recognizedAs = "<uninitialized>"
    
    #print("vt:"+verbText)

    recognizedAs = getRecognizedAs(statement,verbText)
    
    return recognizedAs

In [91]:
def extractTypeOfRecoginition(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    
    typeOfRecognition = "<uninitialized>"
    
    verbTokens = filterVerbTokens(statement)
    
    
    
    for verbToken in verbTokens:
        #print(verbToken.lemma_)
        if (verbToken.lemma_ == 'nominate'):
            typeOfRecognition = 'nomination'
            return typeOfRecognition
        elif (verbToken.lemma_ == 'win'):
            typeOfRecognition = 'winner'
            return typeOfRecognition
        elif (verbToken.lemma_ == 'award'):
            typeOfRecognition = 'winner'
            return typeOfRecognition
        else:
            typeOfRecognition = '<no-match>'
            return typeOfRecognition
    
    return typeOfRecognition

In [63]:
def extractYear(statement):
    doc = nlp(statement)
    for ent in doc.ents:
        if (ent.label_ == "DATE" or ent.label_ == "TIME"):
            return ent.text
    
    return "<no-match>"

In [64]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [65]:
def getNounSubject(statement, verbText):
    doc = nlp(statement)
    for token in doc:
        #print(token.text, token.dep_, token.head.text)
        if((token.dep_ == "nsubjpass" or token.dep_ == "nsubj") and token.head.text == verbText):
            return token.text
    return "<no-match>"

In [66]:
def getMovie(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "nsubjpass" and token.head.text == verbText):
            #print("token nsubjpass : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            #print("token nsubj : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.dep_ == "dobj" and token.head.text == verbText):
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            #print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                #print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                '''
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [67]:
def getWriter(statement, verbText):
    doc = nlp(statement)
    agent = "<uninitialized>" #writer
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
    '''
    
    for token in doc:
        #print("token:",token)
        '''
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (agent) : " + agent)
            return agent
        '''
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            #print("Agent (nsubj) : " + agent)
            return agent
        
        elif((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
        elif(token.head.text == verbText or token.text == verbText):
            # agent = extractOwnerEntitySyntactically(statement)
            agent = getByEntity(statement)
            return agent
        
    return agent

In [68]:
def getByEntity(statement):
    doc = nlp(statement)
    
    for token in doc:
        if token.text == "by":
            for child in token.children:
                if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                    return getORGorPersonThatContainsToken(statement, child)
    return "<no-match>"

In [69]:
def extractOwnerEntitySyntactically(statement):
    doc = nlp(statement)
    verbTokens = filterVerbTokens(statement)
    ORGEntity = []
    
    for token in verbTokens:
        if isExpectedVerbToken(token):
            #print("Expected token is:"+token.text)
            #expected children [was, theatrically, in, on, by]
            byToken = None
            for child in token.children: 
                #usually release location starts with "in"
                if child.text == "by":
                    byToken = child
             
            #print("bytoken : ->", byToken)
            if byToken == None:
                #means no direct by available so lets look at some conj child
                for child in token.children: 
                    #usually release location starts with "in"
                    if child.dep_ == "conj":
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
            
            if byToken != None:
                for subChild in byToken.children:
                    ORGEntity.append(getORGorPersonThatContainsToken(statement, subChild))
                    #print("subchild : " + subChild.text)
                    if (subChild.dep_ == "pobj" and subChild.head.text == "by"):
                        grandChildren = subChild.children
                        childEntity = getORGEntityForGrandChild1(statement, grandChildren, ORGEntity)
                        #print("child entity:    " , str(childEntity))
                        return list(set(childEntity))
                        #ORGEntity.clear()
                        #for item in childEntity:
                            #ORGEntity.append(item)
                    return ORGEntity
    return "<no-match>"

In [70]:
def getORGEntityForGrandChild1(statement, children, ORGEntity):
    '''
    New definition
    '''
    doc = nlp(statement)
    
    for child in children:
        sizeChild = sum(1 for item in child.children)
        #print(child, sizeChild)
        if sizeChild > 0:
            ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
            print(ORGEntity)
            #ORGEntity.append(getORGEntityForGrandChild1(statement, child.children, ORGEntity))
            getORGEntityForGrandChild1(statement, child.children, ORGEntity)
        if sizeChild == 0:
            if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
    
    return ORGEntity

In [71]:
def getRecognizedAs(statement, verbText):
    doc = nlp(statement)
    
    verbTokens = filterVerbTokens(statement)
    
    recognizedAs = "<uninitialized>"
    
    for token in verbTokens:
        if token.lemma_ == "nominate":
            #print(token, token.lemma_)
            recognizedAs = getInfoNominateFor(statement, token.text, doc)
            return recognizedAs
        
        elif token.lemma_ == "win" or token.lemma_ == "award":
            #print(token)
            recognizedAs = getInfoAwardWin(statement, token.text, doc)
            return recognizedAs
        
        else:
            print("no word match")
    
    return "<no-match>"

In [72]:
def getInfoNominateFor(statement, verbText, doc):
    
    nominationFor = "<uninitialized>"
    
    for token in doc:
        if (token.text == "for" and token.head.text == verbText):
            child = [child for child in token.children][0]
            nominationFor = getORGorPersonThatContainsToken(statement, child)
            return nominationFor
    
    return "<no-match>"        

In [73]:
def getInfoAwardWin(statement, verbText, doc):
    awardOf = "<uninitialized>"
    
    for token in doc:
        if (token.dep_ == "dobj" and token.head.text == verbText):
            awardOf = getORGorPersonThatContainsToken(statement, token)
            # print("award of : ", awardOf)
            if (awardOf == "<no-match>"):
                awardOf = getNounChunkThatContainsNoun(statement, token)
            return awardOf
    
    return "<no-match>"

In [74]:
def isExpectedVerbToken(token):
    return (token.lemma_ in ["nominate"] or token.lemma_ in ["win"] or token.lemma_ in ["award"])

In [29]:
def getORGorPersonThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return "<no-match>"

In [30]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]


In [31]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [32]:
def checkRecognizedAsTemplateEligibility(statement):
    print("Matching with wriiten template: for " + statement)
    hasValidVerb = check_verb_match(statement)
    print("Has valid verb:" + str(hasValidVerb))
    
    return hasValidVerb

In [33]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    targetVerbs = ['nominate','win', 'award']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [34]:
def filterVerbTokens(statement):
    doc = nlp(statement)
    result = list(filter(lambda token: token.pos_ == "VERB" and token.lemma_ != "be", doc))
    return result

In [35]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [36]:
get_dependency_tree_nodes('The Mission Impossible was nominated for Golden Globe Award.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'nominated', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'nominated', 'VERB', []],
 ['nominated', 'ROOT', 'nominated', 'VERB', [Impossible, was, for, .]],
 ['for', 'prep', 'nominated', 'VERB', [Award]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'compound', 'Award', 'PROPN', [Golden]],
 ['Award', 'pobj', 'for', 'ADP', [Globe]],
 ['.', 'punct', 'nominated', 'VERB', []]]

In [37]:
get_dependency_tree_nodes('The Mission Impossible was awarded Golden Globe Award.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'awarded', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'awarded', 'VERB', []],
 ['awarded', 'ROOT', 'awarded', 'VERB', [Impossible, was, Award, .]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'compound', 'Award', 'PROPN', [Golden]],
 ['Award', 'dobj', 'awarded', 'VERB', [Globe]],
 ['.', 'punct', 'awarded', 'VERB', []]]

In [38]:
get_dependency_tree_nodes('The Mission Impossible won Golden Globe.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubj', 'won', 'VERB', [The, Mission]],
 ['won', 'ROOT', 'won', 'VERB', [Impossible, Globe, .]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'dobj', 'won', 'VERB', [Golden]],
 ['.', 'punct', 'won', 'VERB', []]]

In [39]:
get_dependency_tree_nodes('The Angry Man was awarded the second-best courtroom drama ever by the American Film Institute.')

[['The', 'det', 'Man', 'PROPN', []],
 ['Angry', 'compound', 'Man', 'PROPN', []],
 ['Man', 'nsubjpass', 'awarded', 'VERB', [The, Angry]],
 ['was', 'auxpass', 'awarded', 'VERB', []],
 ['awarded', 'ROOT', 'awarded', 'VERB', [Man, was, drama, .]],
 ['the', 'det', 'drama', 'NOUN', []],
 ['second', 'advmod', 'best', 'ADJ', []],
 ['-', 'punct', 'best', 'ADJ', []],
 ['best', 'amod', 'drama', 'NOUN', [second, -]],
 ['courtroom', 'compound', 'drama', 'NOUN', []],
 ['drama', 'dobj', 'awarded', 'VERB', [the, best, courtroom, by]],
 ['ever', 'advmod', 'by', 'ADP', []],
 ['by', 'prep', 'drama', 'NOUN', [ever, Institute]],
 ['the', 'det', 'Institute', 'PROPN', []],
 ['American', 'compound', 'Institute', 'PROPN', []],
 ['Film', 'compound', 'Institute', 'PROPN', []],
 ['Institute', 'pobj', 'by', 'ADP', [the, American, Film]],
 ['.', 'punct', 'awarded', 'VERB', []]]

In [40]:
doc = nlp("Dark Knight is directed by Aneesh in his feature debut and written by Chaganty, Sev Ohanian and John.")
for chunk in doc.noun_chunks:
    print(chunk)

Dark Knight
Aneesh
his feature debut
Chaganty
Sev Ohanian
John


In [41]:
doc = nlp ("The Angry Man was awarded the second-best courtroom drama ever by the American Film Institute.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

The Angry Man 0 13 PERSON
second 30 36 ORDINAL
the American Film Institute 66 93 ORG
